## Notes
1. Require config.json

    {
	    "api_key" : "xxxxx",
	    "server" : "https://data.reg.cmu.ac.th"
    }

In [2]:
import sys
!{sys.executable} -m pip install pandas

You should consider upgrading via the 'C:\Users\pruet\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [4]:
import json
import pprint
import urllib
import pandas as pd
import requests

# read configuration
config_file = 'config.json'
config = json.load(open(config_file))
api_key = config['api_key']
server = config['server']


In [16]:
def send_request_to_server(server, api_key, action_path, dataset_dict):
# Make the HTTP request.
	headers_dict = {
		'Authorization': api_key,
		'Content-Type' : 'application/x-www-form-urlencoded'
	}
	if dataset_dict != None:
		dataset_dict = urllib.parse.quote(json.dumps(dataset_dict))
	else:
		dataset_dict = []
	req = requests.post(server + 'api/action/' + action_path, headers=headers_dict, data=dataset_dict)
	
	# Use the json module to load CKAN's response into a dictionary.
	response_dict = req.json()

	# Check the contents of the response.
	#assert response_dict['success'] is True
	#result = response_dict['result']
	pprint.pprint(response_dict)
	return response_dict


In [6]:
def create_package(server, api_key, dataset_dict):
	send_request_to_server(server, api_key, 'package_create', dataset_dict)

In [7]:
def update_package(server, api_key, dataset_dict):
	send_request_to_server(server, api_key, 'package_update', dataset_dict)

In [21]:
def get_package_info(server, api_key, package_name):
	return send_request_to_server(server, api_key, 'package_show?id=' + package_name, None)

In [46]:
def get_resource_search_name_name(server, api_key, resource_name):
	response = send_request_to_server(server, api_key, 'resource_search?query=name:' + resource_name, None)
	if response['result']['count'] > 0:
		return response['result']['results'][0]
	return None

In [48]:
result = get_resource_search_name_name(server, api_key, 'Grduate Student (Excel)')
print(result)

{'help': 'http://10.110.2.208/api/3/action/help_show?name=resource_search',
 'result': {'count': 1,
            'results': [{'cache_last_updated': None,
                         'cache_url': None,
                         'created': '2021-12-01T04:53:46.727283',
                         'description': 'Test employee data',
                         'format': 'XLS',
                         'hash': '',
                         'id': '5b3d89bc-0e97-4ed5-8367-5d77bd29f101',
                         'last_modified': None,
                         'metadata_modified': '2021-12-01T04:53:46.715254',
                         'mimetype': None,
                         'mimetype_inner': None,
                         'name': 'Grduate Student (Excel)',
                         'package_id': '534e333c-8a57-418e-af24-095c1a84277e',
                         'position': 0,
                         'resource_type': None,
                         'size': None,
                         'state': 'active',

In [22]:
get_package_info(server, api_key, 'graduatestudent')

{'help': 'http://10.110.2.208/api/3/action/help_show?name=package_show',
 'result': {'author': 'Pruet Boonma',
            'author_email': 'pruet.b@cmu.ac.th',
            'creator_user_id': '71acce18-1a54-43b2-adb7-23a0494312f6',
            'extras': [{'key': 'AdvisorAccess', 'value': 'OwnAdvisee'},
                       {'key': 'DepartmentAccess', 'value': 'OwnDepartment'},
                       {'key': 'FacultyAccess', 'value': 'OwnFaculty'},
                       {'key': 'PDPA', 'value': 'Sensitive'},
                       {'key': 'PublicAccess', 'value': 'No'},
                       {'key': 'RetainPolicy', 'value': 'Deleted when retired'},
                       {'key': 'StudentAccess', 'value': 'OwnRecord'},
                       {'key': 'UniversityAccess', 'value': 'All'},
                       {'key': 'UpdatePolicy', 'value': 'NaN'}],
            'groups': [],
            'id': '534e333c-8a57-418e-af24-095c1a84277e',
            'isopen': False,
            'license_id'

{'help': 'http://10.110.2.208/api/3/action/help_show?name=package_show',
 'success': True,
 'result': {'author': 'Pruet Boonma',
  'author_email': 'pruet.b@cmu.ac.th',
  'creator_user_id': '71acce18-1a54-43b2-adb7-23a0494312f6',
  'id': '534e333c-8a57-418e-af24-095c1a84277e',
  'isopen': False,
  'license_id': None,
  'license_title': None,
  'maintainer': 'Pruet Boonma',
  'maintainer_email': 'pruet.b@cmu.ac.th',
  'metadata_created': '2021-11-30T16:53:46.699055',
  'metadata_modified': '2021-12-01T04:04:23.138235',
  'name': 'graduatestudent',
  'notes': 'Record of student',
  'num_resources': 0,
  'num_tags': 2,
  'organization': {'id': '12b44fd7-a60c-4f0a-a0a0-254f7b307ff3',
   'name': 'registrationoffice',
   'title': 'RegistrationOffice',
   'type': 'organization',
   'description': '',
   'image_url': '',
   'created': '2021-11-30T16:11:00.017034',
   'is_organization': True,
   'approval_status': 'approved',
   'state': 'active'},
  'owner_org': '12b44fd7-a60c-4f0a-a0a0-254f7b3

In [34]:
def create_resource(server, api_key, dataset_dict):
	package_info = get_package_info(server, api_key, dataset_dict['name'])
	package_id = package_info['result']['id']
	dataset_dict_resource = {
		'package_id' : package_id,
		'url' : dataset_dict['resource_url'],
		'name' : dataset_dict['resource_name'],
		'description' : dataset_dict['resource_description'],
		'format' : dataset_dict['resource_format']
	}
	resource = get_resource_search_name_name(server, api_key, dataset_dict['resource_name'])
	if resource == None:
		send_request_to_server(server, api_key, 'resource_create', dataset_dict_resource)
	else:
		dataset_dict_resource['id'] = resource['id']
		send_request_to_server(server, api_key, 'resource_update', dataset_dict_resource)

In [37]:
df = pd.read_csv('ckan_data.csv')
for index, row in df.iterrows():
	tags = row['Tags'].split(',')
	tag_list= []
	for tag in tags:
		tag_list.append({'name' : tag})
	dataset_dict = {
		'owner_org' : row['Organization'],
		'groups' : [
			{'id' : row['Group']}
		],
		'name' : row['Name'],
		'title' : row['Title'],
		'author' : row['Author'],
		'notes' : row['Notes'],
		'author_email' : row['AuthorEmail'],
		'maintainer' : row['Maintainer'],
		'maintainer_email' : row['MaintainerEmail'],
		'url' : row['Url'],
		'tags' : tag_list,
		'resource_name' : row['ResourceName'],
		'resource_url' : row['ResourceUrl'],
		'resource_format' : row['ResourceFormat'],
		'resource_description' : row['ResourceDescripiton'],
		'extras' : [
			{'key' : 'PublicAccess', 'value' : row['PublicAccess']},
			{'key' : 'UniversityAccess', 'value' : row['UniversityAccess']},
			{'key' : 'FacultyAccess', 'value' : row['FacultyAccess']},
			{'key' : 'DepartmentAccess', 'value' : row['DepartmentAccess']},
			{'key' : 'AdvisorAccess', 'value' : row['AdvisorAccess'] },
			{'key' : 'StudentAccess', 'value' : row['StudentAccess']},
			{'key' : 'PDPA', 'value' : row['PDPA']},
			{'key' : 'RetainPolicy', 'value' : row['ReatainPolicy'] },
			{'key' : 'UpdatePolicy', 'value' : row['UpdatePolicy'] }
		]
	}
	create_package(server, api_key, dataset_dict)
	create_resource(server, api_key, dataset_dict)

{'error': {'__type': 'Validation Error',
           'name': ['That URL is already in use.']},
 'help': 'http://10.110.2.208/api/3/action/help_show?name=package_create',
 'success': False}
{'help': 'http://10.110.2.208/api/3/action/help_show?name=package_show',
 'result': {'author': 'Pruet Boonma',
            'author_email': 'pruet.b@cmu.ac.th',
            'creator_user_id': '71acce18-1a54-43b2-adb7-23a0494312f6',
            'extras': [{'key': 'AdvisorAccess', 'value': 'OwnAdvisee'},
                       {'key': 'DepartmentAccess', 'value': 'OwnDepartment'},
                       {'key': 'FacultyAccess', 'value': 'OwnFaculty'},
                       {'key': 'PDPA', 'value': 'Sensitive'},
                       {'key': 'PublicAccess', 'value': 'No'},
                       {'key': 'RetainPolicy', 'value': 'NaN'},
                       {'key': 'StudentAccess', 'value': 'OwnRecord'},
                       {'key': 'UniversityAccess', 'value': 'All'},
                       {'key': 